In [1]:
from sqlite3 import connect
import numpy as np
import pandas as pd 
from sqlalchemy import create_engine



In [2]:

# conn = connect(pathTo)
# query ='SELECT pitch FROM m_thread_tolerance'
# df_sql = pd.read_sql(query, conn)


def get_df_from_sql(path,query_sql):
    conn = connect(path)
    return pd.read_sql(query_sql, conn)
    




def get_patch_to(file):
    return '../db/'+file+'.db'

def get_patch_from(file):
    return '../db/'+file+'.db'

def getSplitValue(listNameColumns, indexSplit):
    listSplitValue = []
   
    for nameColumn in listNameColumns:
        listSplitValue.append(nameColumn.split('_')[indexSplit])
    return listSplitValue

         

# tr

In [3]:
file = 'ThreadDB___'

# ******************************
patchFrom = get_patch_from(file)


df = get_df_from_sql(patchFrom,'SELECT * FROM Tr')


In [4]:
df_rename_column_m = df.iloc[:,7:49]

In [5]:
listNameColumns = list(df_rename_column_m.columns)

esEiList = getSplitValue(listNameColumns,0)
dList = getSplitValue(listNameColumns,1)
toleranceList = getSplitValue(listNameColumns,2)
# mList = getSplitValue(listNameColumns,3)


newListNameColumns =[]

for i in range(len(toleranceList)):
    newListNameColumns.append(   dList[i] +"_"+  esEiList[i]+"_"+toleranceList[i]+"_m")


df_rename_column_m.columns = newListNameColumns

In [6]:


df_new =  df.iloc[:,:7]



In [7]:
df_new = pd.concat([df_new, df_rename_column_m], axis=1)

In [8]:
df_rename_column_f = df.iloc[:, 49:]

listNameColumns = list(df_rename_column_f.columns)

esEiList = getSplitValue(listNameColumns,0)
dList = getSplitValue(listNameColumns,1)
toleranceList = getSplitValue(listNameColumns,2)
mList = getSplitValue(listNameColumns,3)


newListNameColumns =[]

for i in range(len(toleranceList)):
    newListNameColumns.append(  ( dList[i] +"_"+  esEiList[i]+"_"+toleranceList[i]+"_f").casefold())


df_rename_column_f.columns = newListNameColumns

df_new = pd.concat([df_new, df_rename_column_f], axis=1)

In [9]:
df_to_float1 =  df.iloc[:,:7]
df_to_float2 = df_new.iloc[:, 7:]

In [10]:

df_to_float2 = df_to_float2.astype(float)
df_new_float = pd.concat([df_to_float1, df_to_float2], axis=1)

In [11]:
df_new_float.dtypes

_id                   int64
nominal_diameter      int64
pitch               float64
D4                  float64
d2_D2               float64
                     ...   
d4_ei_9h_f          float64
d2_es_9h_f          float64
d2_ei_9h_f          float64
d1_es_9h_f          float64
d1_ei_9h_f          float64
Length: 69, dtype: object

In [12]:
engine = create_engine('sqlite:///../db/thread.db', echo=False)
df_new_float.to_sql('tr', con=engine, if_exists='replace', index=False)

222